## Hereditary Geometry Discovery

- Simultaneously learn:
    - 1. Tensor that aligns Kernels (the left-action).
    - 2. Learn generator W that contains log of left-actions.
- Not implemented yet:
    - 1. Charts
    - 2. Symmetry Penalizer

In [1]:
import pickle

from src.utils import load_replay_buffer
from src.learning.symmetry_discovery.differential.kernel_approx import KernelFrameEstimator

TASK_0_NAME="sac_circle_rotation_task_0"
TASK_1_NAME="sac_circle_rotation_task_1"
TASK_2_NAME="sac_circle_rotation_task_2"

LOAD_WHAT:str="next_observations"
KERNEL_DIM=1
N_SAMPLES=50_000


def load_replay_buffer_and_kernel(task_name:str, load_what:str, kernel_dim: int, n_samples:int):
    """Loads samples and kernel evaluator of a task."""

    assert load_what in ["observations", "actions", "next_observations"], "Learn hereditary geometry for states, actions or next states."

    buffer_name= f"{task_name}_replay_buffer.pkl"
    kernel_name= f"{task_name}_kernel_bases.pkl"

    buffer= load_replay_buffer(buffer_name, N_steps=n_samples)
    ps=buffer[load_what]
    print(f"Loaded {load_what} from {buffer_name} with shape {ps.shape}")

    # Load kernel bases
    frameestimator=KernelFrameEstimator(ps=ps, kernel_dim=kernel_dim)
    with open(kernel_name, 'rb') as f:
        kernel_samples = pickle.load(f)
    frameestimator.set_frame(frame=kernel_samples)

    return ps, frameestimator

ps_task_0, frameestimator_task_0 = load_replay_buffer_and_kernel(TASK_0_NAME, "next_observations", kernel_dim=KERNEL_DIM, n_samples=N_SAMPLES)
ps_task_1, frameestimator_task_1 = load_replay_buffer_and_kernel(TASK_1_NAME, "next_observations", kernel_dim=KERNEL_DIM, n_samples=N_SAMPLES)
ps_task_2, frameestimator_task_2 = load_replay_buffer_and_kernel(TASK_2_NAME, "next_observations", kernel_dim=KERNEL_DIM, n_samples=N_SAMPLES)

2025-07-02 13:56:15.584036: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.
INFO:faiss:Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes. This is only an error if you're trying to use GPU Faiss.
/Users/paulnitschke/Desktop/projects/geo_meta_rl/src/utils.py:68: UserWarning: Replay buffer contains more samples than selected.
  warnings.warn("Replay buffer contains more samples than selected.")


Loaded next_observations from sac_circle_rotation_task_0_replay_buffer.pkl with shape torch.Size([100000, 2])


INFO:root:Setup kernel frame evaluation with k-NN index.


Loaded next_observations from sac_circle_rotation_task_1_replay_buffer.pkl with shape torch.Size([100000, 2])


INFO:root:Setup kernel frame evaluation with k-NN index.


Loaded next_observations from sac_circle_rotation_task_2_replay_buffer.pkl with shape torch.Size([100000, 2])


INFO:root:Setup kernel frame evaluation with k-NN index.


In [2]:
from src.learning.symmetry_discovery.differential.hereditary_geometry_discovery import HereditaryGeometryDiscovery


tasks_ps= [ps_task_0, ps_task_1, ps_task_2]
tasks_frameestimators= [frameestimator_task_0, frameestimator_task_1, frameestimator_task_2]
her_geo_dis=HereditaryGeometryDiscovery(tasks_ps=tasks_ps,tasks_frameestimators=tasks_frameestimators, kernel_dim=KERNEL_DIM, batch_size=64, knn=10, bandwidth=0.5)
her_geo_dis.optimize(n_steps=1000)

/Users/paulnitschke/Desktop/projects/geo_meta_rl/src/learning/symmetry_discovery/differential/hereditary_geometry_discovery.py:34: UserWarning: Only one left-action implemented.
  warnings.warn("Only one left-action implemented.")
/Users/paulnitschke/Desktop/projects/geo_meta_rl/src/learning/symmetry_discovery/differential/hereditary_geometry_discovery.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.lg_i=torch.tensor(torch.randn(self.n, self.n), requires_grad=True)
Inferring left-action:   0%|          | 0/1000 [00:00<?, ?it/s]


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.